In [39]:
# https://www.kaggle.com/c/text-normalization-challenge-english-language

In [21]:
import pickle as pkl
import numpy as np
from pathlib import Path
import csv
from sklearn.model_selection import train_test_split

In [26]:
PATH = Path('DATA')
x, y = [], []

### Read data

In [27]:
with open(PATH/'kaggle_data/en_train.csv', newline='', encoding='utf8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for i, row in enumerate(spamreader):
        if i > 0:
            x.append(row[3])
            y.append(row[4])

print('Num examples:', len(x), len(y))
print('Num classes: ', len(set(y)))


x_trn, x_val, y_trn, y_val = train_test_split(x, y, test_size=0.05, random_state=42)
pkl.dump([x_trn, y_trn, x_val, y_val], open(PATH/'xt_yt_xv_yv.pkl', 'wb'))

Num examples: 9918441 9918441
Num classes:  471587


### Load data

In [32]:
NUM_CLASSES = 471587
with open(PATH/'xt_yt_xv_yv.pkl', 'rb') as f:
    x_trn, y_trn, x_val, y_val = pkl.load(f)

#### Inspect data

In [38]:
v = 50
print('\n############## Train data:')
for i in range(v):
    print(x_trn[i], '-', y_trn[i])
    
print('\n############## Val data:')
for i in range(v):
    print(x_val[i], '-', y_val[i])


############## Train data:
, - ,
is - is
then - then
D - D
Mark - Mark
: - :
1914 - nineteen fourteen
; - ;
is - is
" - "
in - in
left - left
. - .
: - :
and - and
The - The
, - ,
seat - seat
into - into
commission - commission
Kickstarter - Kickstarter
viewed - viewed
books - books
survival - survival
A - A
they - they
cities - cities
. - .
Veil - Veil
Hall - Hall
realises - realizes
in - in
in - in
) - )
The - The
RT - r t
. - .
area - area
17 - seventeen
Aimée - a i m e acute e
left - left
. - .
up - up
機 - 機
- - -
Sculpture - Sculpture
chapter - chapter
Rose - Rose
pink - pink
with - with

############## Val data:
, - ,
, - ,
signed - signed
semifinalists - semifinalists
Weemie - Weemie
LED - LED
squirrels - squirrels
third - third
though - though
he - he
Parliament - Parliament
Fungorum - Fungorum
although - although
. - .
" - "
, - ,
. - .
seven - seven
Lecithoceridae - Lecithoceridae
Spaniards - Spaniards
spending - spending
1073 - ten seventy three
The - The
of - of
, - ,
Live

#### Tokenize data

In [ ]:
# TODO